In [1]:
import pandas as pd
import zipfile
import os
import requests
from Levenshtein import distance
import time
import numpy as np
from collections import Counter

In [2]:
from buscadores2 import BuscadorDistancias

In [3]:
buscador=BuscadorDistancias(distance)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/github/correcci-n_nombres'

In [12]:
#Establecer directorio de trabajo
os.chdir("D:/github/correcci-n_nombres"),
#os.chdir("C:/Users/claud/Documents/GitHub/correcci-n_nombres")

(None,)

In [13]:
"#Url de descarga de archivo zip",
#url="https://pub.bienestar.gob.mx/data/v2/last/S052/22A_22A/Programa_S052_S052_22A_22A.zip",
url="https://pub.bienestar.gob.mx/data/v2/last/S072/225_226/Programa_S072_S072_225_226.zip"

In [14]:
#Descargar archivo zip"
r = requests.get(url,verify=False)
with open("Programa_S072_S072_225_226.zip", "wb") as code:
        code.write(r.content)

c:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pub.bienestar.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [15]:
#Descomprimir archivo zip
with zipfile.ZipFile("Programa_S072_S072_225_226.zip","r") as zip_ref:
        zip_ref.extractall("raw_data")
os.remove("Programa_S072_S072_225_226.zip")

In [16]:
#Detectar todos los archivos csv de la carpeta csv dentro del directorio de trabajo",
csv_files = [f for f in os.listdir("raw_data/csv") if f.endswith('.csv')]

In [17]:
#Leer todos los archivos csv y concatenarlos en un solo dataframe"
df = pd.concat((pd.read_csv("raw_data/csv/"+f, encoding='utf-8', sep=',', low_memory=False) for f in csv_files))
#Columnas en minuscúlas",
df.columns = map(str.lower, df.columns)
#Seleccionar columnas 3 a 5"
df = df.iloc[:, 3:6]
#Renombrar columnas"
df.columns = ["primer_apellido", "segundo_apellido", "nombre"]
#Eliminar los registros "MENOR" en las tres columnas usando un loop"
for i in df.columns:
    df = df[df[i] != "MENOR"]
    #Eliminar nans de todas las columnas",
    df = df.dropna()
#Total de registros"
print("Total de registros:", len(df))

Total de registros: 2028974


In [18]:
#Dejar únicamente el primer nombre"
df["nombre"] = df["nombre"].str.split(" ").str[0]

In [19]:
#Contar si existen nans en las columnas"
df.isna().sum()

primer_apellido     0
segundo_apellido    0
nombre              0
dtype: int64

In [20]:
#Crear una lista de apellidos y nombres"
apellidos = df.primer_apellido.tolist() + df.segundo_apellido.tolist()
nombres = df.nombre.tolist()

#Eliminar registros que se repitan 3 veces o menos"
freq_apellidos = Counter(apellidos)
freq_nombres = Counter(nombres)
apellidos = [x for x in apellidos if freq_apellidos[x] >= 2]
nombres = [x for x in nombres if freq_nombres[x] >= 2]
#Dejar valores únicos
apellidos = list(set(apellidos))
nombres = list(set(nombres))

#Hacer una lista general de nombres y apellidos
nombres_apellidos = apellidos + nombres

#Ordenar listas
apellidos.sort()
nombres.sort()
nombres_apellidos.sort()
print("Número de registros en la lista: ", len(nombres_apellidos))

Número de registros en la lista:  29756


In [21]:
#Cargar archivo de excel con nombres a corregir
nombres_corregir = pd.read_excel("Nombres_SEMEFO_Anotaciones.xlsx")
#Dejar únicamente la variable FourthConfig
nombres_corregir = nombres_corregir.iloc[:,[4]]
#Eliminar NaNs
nombres_corregir = nombres_corregir.dropna()
#Separar FourthConfig en el mayor número de columnas posibles
nombres_corregir = nombres_corregir["FourthConfig"].str.split(" ", expand = True)
#Dejar solo las primeras 3 columnas 
nombres_corregir = nombres_corregir.iloc[:,[0,1,2]]
#Renombrar columnas"
nombres_corregir.columns = ["primer_apellido", "segundo_apellido", "nombre"]

In [23]:
start_time = time.perf_counter()

#Iterar sobre cada fila de la columna nombre_mal\n",
for i in nombres_corregir["primer_apellido"]:
    #Obtener la distancia de Levenshtein entre el nombre a corregir y cada uno de los nombres de la lista"

    distancias = [distance(i, j) for j in nombres_apellidos]
    #Obtener el índice de los 3 nombres más cercanos"
    indices = np.argsort(distancias)[:3]
    #Obtener los 3 nombres más cercanos
    nombres_mas_cercanos = [nombres_apellidos[i] for i in indices]
    #Crear 3 columnas con los 3 nombres más cercanos
    nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_1"] = nombres_mas_cercanos[0]
    #nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_2"] = nombres_mas_cercanos[1]
    #nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_3"] = nombres_mas_cercanos[2]
    #Ordenar por primer_apellido"
    nombres_corregir = nombres_corregir.sort_values(by = "primer_apellido")
    
end_time = time.perf_counter()

#Imprimir el tiempo de ejecución\n",
print("Número de elementos revisados: ", len(nombres_corregir))
print(f'Tiempo de ejecución: {end_time - start_time} segundos')
print("Tiempo por elemento: ", (end_time - start_time)/len(nombres_corregir), " segundos")

Número de elementos revisados:  257
Tiempo de ejecución: 6.321946400001252 segundos
Tiempo por elemento:  0.024599013229576857  segundos


In [24]:
pd.set_option('display.max_rows', None)
nombres_corregir

,primer_apellido,segundo_apellido,nombre,nombre_cercano_1
152,2,PABLO,MARTINEZ,A
198,2,E,RAMA,A
85,28404819983,0,None,A
52,A,A,None,A
217,A,SCULINO,DE,A
205,A,MESE,A,A
116,A,O,A,A
27,A,A,A,A
197,A,CASTILLO,CAMPOS,A
252,A,VALQUEZ,GALLARDO,A
